## 🧾 Introduction
This notebook demonstrates how to build a Retrieval-Augmented Generation (RAG) based Question-Answering (QA) system for a business using OpenAI's LLM and Pinecone vector database. The pipeline ingests business documents, indexes them using vector embeddings, and responds to user questions with context-aware answers.

SECTION 1: Setup Environment (Install Required Packages)


In [ ]:
!pip install -q openai langchain pinecone python-dotenv unstructured pdfminer.six faiss-cpu sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 14.0 MB/s eta 0:00:00


SECTION 2: Import Libraries


In [ ]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.7 MB/s eta 0:00:00


In [ ]:
import os
import openai
import pinecone
from langchain.vectorstores import Pinecone
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain_community.document_loaders import UnstructuredFileLoader



API Key Setup


In [ ]:
OPENAI_API_KEY = "openai_api_key"
PINECONE_API_KEY = "pinecone_api_key"
PINECONE_ENVIRONMENT = "gcp-starter"
PINECONE_INDEX_NAME = "business-index"
# Set OpenAI key
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
openai.api_key = OPENAI_API_KEY

SECTION 4: Load Docs + Preprocess (Split into Chunks)

In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload your DOCX/PDF file

file_name = list(uploaded.keys())[0]  # Get uploaded file name

from langchain_community.document_loaders import UnstructuredFileLoader
doc_loader = UnstructuredFileLoader(file_name)

raw_docs = doc_loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = splitter.split_documents(raw_docs)

print(f"Loaded {len(documents)} chunks")


Saving CS 3.pdf to CS 3 (4).pdf
Loaded 176 chunks


SECTION 5: Embed Documents & Upload to Pinecone

In [ ]:
import os
from pinecone import Pinecone as PineconeClient, ServerlessSpec
from langchain.vectorstores import Pinecone as PineconeVectorStore
from langchain.embeddings import OpenAIEmbeddings

# Initialize Pinecone client
pc = PineconeClient(api_key=PINECONE_API_KEY)
embeddings = OpenAIEmbeddings()

# Check if the index exists, and create it if it doesn't
if PINECONE_INDEX_NAME not in pc.list_indexes().names():
    pc.create_index(
        name=PINECONE_INDEX_NAME,
        dimension=1536,  # This is the dimensionality of OpenAI's Ada-002 embeddings
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1' # Replace with your desired region
        )
    )

# Get the Pinecone index object
pinecone_index = pc.Index(PINECONE_INDEX_NAME)

# Generate embeddings and prepare data for upsert
vectors = embeddings.embed_documents([doc.page_content for doc in documents])
upsert_data = []
for i, doc in enumerate(documents):
    upsert_data.append((f"doc_{i}", vectors[i], {"text": doc.page_content})) # Using index as ID, add metadata

# Upsert data to Pinecone
batch_size = 100
for i in range(0, len(upsert_data), batch_size):
    batch = upsert_data[i:i + batch_size]
    pinecone_index.upsert(vectors=batch)

print(f"Upserted {len(upsert_data)} vectors to Pinecone.")

# Initialize PineconeVectorStore from the existing index for retrieval
vectorstore = PineconeVectorStore.from_existing_index(
    index_name=PINECONE_INDEX_NAME,
    embedding=embeddings
)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

SECTION 6: Query Answering using RAG (Retriever + LLM)

In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo")
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)

/tmp/ipython-input-44-555743000.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-3.5-turbo")


NameError: name 'vectorstore' is not defined

SECTION 7: Test Output (Ask a Question)

In [ ]:
query = "What services does our business offer?"
result = qa_chain.invoke(query)
print("\nAnswer:\n", result['result'])
print("\nSource Document Snippets:\n")
for doc in result['source_documents']:
    print(doc.page_content[:300], "...\n")


In [ ]:
!pip install "unstructured[pdf]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.9/527.9 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.6 MB/s eta 0:00:00
